In [1]:
import cv2
import numpy as np
import os

class SmartTextScanner:
    def __init__(self, reference_image_path, roi_coords):
        """
        Ініціалізація сканера. Це робиться ОДИН РАЗ при старті роботи з новим типом екрану.

        :param reference_image_path: Шлях до "Ідеального" зображення (clean_dataset)
        :param roi_coords: Кортеж (x1, y1, x2, y2) - зона тексту на ідеальному зображенні
        """
        self.ref_img = cv2.imread(reference_image_path)
        if self.ref_img is None:
            raise ValueError(f"Не вдалося завантажити еталон: {reference_image_path}")

        # Зберігаємо координати зони
        self.roi = roi_coords # (x1, y1, x2, y2)

        # Ініціалізуємо детектор ознак (SIFT - найнадійніший для цього)
        # Якщо SIFT повільний, можна замінити на cv2.ORB_create()
        self.detector = cv2.SIFT_create()

        # Знаходимо ключові точки на ЕТАЛОНІ один раз і запам'ятовуємо їх
        self.kp_ref, self.des_ref = self.detector.detectAndCompute(self.ref_img, None)

        # Матчер для порівняння точок (FLANN або BFMatcher)
        self.matcher = cv2.BFMatcher()

        print(f"Сканер ініціалізовано. Знайдено {len(self.kp_ref)} точок на еталоні.")

    def extract_text_zone(self, query_image_path, visualize=False):
        """
        Головний метод. Отримує криве фото, знаходить зону і вирізає її.

        :param query_image_path: Шлях до фото, яке зробив користувач
        :param visualize: Чи зберігати налагоджувальну картинку з рамками
        :return: Вирівняне зображення зони тексту (numpy array) або None
        """
        # 1. Завантажуємо поточне фото
        img_query = cv2.imread(query_image_path)
        if img_query is None:
            print("Помилка завантаження фото користувача.")
            return None

        # 2. Знаходимо точки на новому фото
        kp_query, des_query = self.detector.detectAndCompute(img_query, None)
        if des_query is None or len(kp_query) < 4:
            print("Занадто мало деталей на фото для прив'язки.")
            return None

        # 3. Співставляємо точки з еталоном
        matches = self.matcher.knnMatch(self.des_ref, des_query, k=2)

        # Фільтрація Lowe's ratio test (відсіюємо слабкі співпадіння)
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

        if len(good_matches) < 10: # Треба хоча б 10 хороших точок
            print("Не вдалося знайти еталонний екран на фото.")
            return None

        # 4. Обчислюємо Гомографію (Матрицю спотворень)
        src_pts = np.float32([self.kp_ref[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_query[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        if M is None:
            print("Не вдалося розрахувати геометрію сцени.")
            return None

        # 5. Трансформуємо координати зони
        # Беремо початковий прямокутник
        x1, y1, x2, y2 = self.roi
        h_roi = y2 - y1
        w_roi = x2 - x1
        pts_ref = np.float32([ [x1, y1], [x2, y1], [x2, y2], [x1, y2] ]).reshape(-1, 1, 2)

        # Застосовуємо матрицю M, щоб знайти, де ці точки на кривому фото
        dst_roi_corners = cv2.perspectiveTransform(pts_ref, M)

        # 6. "Вирівнювання" (Warp Perspective)
        # Ми хочемо отримати ідеальний прямокутник розміру w_roi x h_roi
        dst_pts_flat = np.float32([ [0, 0], [w_roi, 0], [w_roi, h_roi], [0, h_roi] ])

        # Матриця для вирізання (від кривих координат до плоского прямокутника)
        M_warp = cv2.getPerspectiveTransform(dst_roi_corners, dst_pts_flat)

        # Вирізаємо і вирівнюємо
        cropped_zone = cv2.warpPerspective(img_query, M_warp, (int(w_roi), int(h_roi)))

        # --- Візуалізація (для перевірки) ---
        if visualize:
            debug_img = img_query.copy()
            cv2.polylines(debug_img, [np.int32(dst_roi_corners)], True, (0, 255, 0), 3, cv2.LINE_AA)
            save_path = "debug_scan_result.jpg"
            cv2.imwrite(save_path, debug_img)
            print(f"Збережено візуалізацію в {save_path}")

        return cropped_zone

# --- ПРИКЛАД ВИКОРИСТАННЯ В ПАЙПЛАЙНІ ---

# 1. Симуляція: Дані, які ми "запам'ятали" колись давно
REF_IMAGE = "clean_dataset/images/000000.png"
# Наприклад, користувач виділив зону [x1, y1, x2, y2]
# Я беру ці дані з файлу для прикладу, але у вас це буде з бази даних
with open("test/labels/000000.txt", "r") as f:
    # Припустимо формат: x1 y1 x2 y2
    parts = f.read().strip().split()
    saved_roi = tuple(map(float, parts[-4:])) # (x1, y1, x2, y2)

print(f"Завантажуємо 'Чорний ящик' з еталоном: {REF_IMAGE}")
print(f"Збережена зона: {saved_roi}")

# Ініціалізація
scanner = SmartTextScanner(REF_IMAGE, saved_roi)

# 2. Симуляція: Прийшло "криве" фото від користувача
USER_PHOTO = "output/images/000000_006.png" # Якесь випадкове з папки

print(f"Отримано фото користувача: {USER_PHOTO}")
result_image = scanner.extract_text_zone(USER_PHOTO, visualize=True)

if result_image is not None:
    print("Успіх! Зона знайдена і вирізана.")

    # 3. Передача в YOLO + Classifier (Симуляція)
    # Тут ви вже передаєте `result_image` у ваші моделі

    # cv2.imshow("Result for Model", result_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # Збережемо результат, щоб ви побачили
    cv2.imwrite("final_extracted_zone.png", result_image)
    print("Результат збережено як final_extracted_zone.png")
else:
    print("Не вдалося обробити зображення.")

Завантажуємо 'Чорний ящик' з еталоном: clean_dataset/images/000000.png
Збережена зона: (539.0, 368.0, 650.0, 488.0)
Сканер ініціалізовано. Знайдено 412 точок на еталоні.
Отримано фото користувача: output/images/000000_006.png
Збережено візуалізацію в debug_scan_result.jpg
Успіх! Зона знайдена і вирізана.
Результат збережено як final_extracted_zone.png
